# Machine Learning Model Deployment: Predict if Song will be Liked by Me

## Trainer Script

In [2]:
!python train.py

01:28:30 INFO:Loading dataset
01:28:31 INFO:Setting column `is_liked` as target column
01:28:31 INFO:Removing records that has n/a value ...
01:28:31 INFO:Performing imbalance class handling by oversampling it (using SMOTENC) and downsampling it using RandomUnderSampler
01:28:44 INFO:Non used feature for this training session is: ['track_id', 'album_name', 'track_name', 'time_signature']
01:28:44 INFO:Preparing the ready to use dataset
01:28:44 INFO:Performing train test split with ratio of 60/20/20
01:28:44 INFO:Converting train/test/val feature into dictvectorizer
01:28:44 INFO:Training LightGBMClassifier Model
[LightGBM] [Info] Number of positive: 3665, number of negative: 3595
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001766 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3521
[LightGBM] [Info] Number of data points in the tra

## Preparing the Serving Script

In [3]:
!pip install pipenv

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 2.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 2.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.9/468.9 kB 2.4 MB/s eta 0:00:00a 0:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [5]:
!~/.local/bin/pipenv install

Pipfile.lock not found, creating...
Locking [packages] dependencies...
Building requirements...
Resolving dependencies...
✔ Success! Locking...
⠧ Locking...
Locking [dev-packages] dependencies...
Updated Pipfile.lock (44ce764afc73e947adebbf0fd95999fb81e30a113b55c8b6b9c7bea27ead5b03)!
Installing dependencies from Pipfile.lock (ad5b03)...
To activate this project's virtualenv, run pipenv shell.
Alternatively, run a command inside the virtualenv with pipenv run.


## Taking some of the data samples for demo

Composition of data to be taken:
1. two songs that available in my personal liked songs
2. two songs that not available in my personal liked songs, but I liked it
3. four songs that not available in my personal liked songs, but I dont liked it

In [11]:
import pandas as pd

data = pd.read_csv('data/labeled_dataset.csv').drop(columns=['Unnamed: 0', 'Unnamed: 0.1'])
data

,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre,is_liked
0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,1,...,0,0.1430,0.0322,0.000001,0.3580,0.7150,87.917,4,acoustic,1
1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,1,...,1,0.0763,0.9240,0.000006,0.1010,0.2670,77.489,4,acoustic,0
2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,0,...,1,0.0557,0.2100,0.000000,0.1170,0.1200,76.332,4,acoustic,0
3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,0,...,1,0.0363,0.9050,0.000071,0.1320,0.1430,181.740,3,acoustic,0
4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,2,...,1,0.0526,0.4690,0.000000,0.0829,0.1670,119.949,4,acoustic,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113995,2C3TZjDRiAzdyViavDJ217,Rainy Lullaby,#mindfulness - Soft Rain for Mindful Meditatio...,Sleep My Little Boy,21,384999,False,0.172,0.2350,5,...,1,0.0422,0.6400,0.928000,0.0863,0.0339,125.995,5,world-music,0
113996,1hIz5L4IB9hN3WRYPOCGPw,Rainy Lullaby,#mindfulness - Soft Rain for Mindful Meditatio...,Water Into Light,22,385000,False,0.174,0.1170,0,...,0,0.0401,0.9940,0.976000,0.1050,0.0350,85.239,4,world-music,0
113997,6x8ZfSoqDjuNa5SVP5QjvX,Cesária Evora,Best Of,Miss Perfumado,22,271466,False,0.629,0.3290,0,...,0,0.0420,0.8670,0.000000,0.0839,0.7430,132.378,4,world-music,0
113998,2e6sXL2bYv4bSz6VTdnfLs,Michael W. Smith,Change Your World,Friends,41,283893,False,0.587,0.5060,7,...,1,0.0297,0.3810,0.000000,0.2700,0.4130,135.960,4,world-music,0


In [24]:
liked_song = data[data['is_liked'] == 1].sample(2, random_state=42).to_dict('records')
personally_liked = data[data['track_id'].map(lambda x: x in ('4BE1OloRc9xwjyqA4wFFuN', '0j2WBxWZnWti5TpSxjJvPb'))].to_dict('records')
non_liked_song = data[data['is_liked'] == 0].sample(4, random_state=42).to_dict('records')
samples = liked_song + personally_liked + non_liked_song

samples

[{'track_id': '0ett0qCHbl0vdSRFhRSkMc',
  'artists': 'OneRepublic',
  'album_name': 'Native',
  'track_name': 'If I Lose Myself',
  'popularity': 53,
  'duration_ms': 241560,
  'explicit': False,
  'danceability': 0.485,
  'energy': 0.699,
  'key': 2,
  'loudness': -6.017,
  'mode': 1,
  'speechiness': 0.0479,
  'acousticness': 0.096,
  'instrumentalness': 0.0,
  'liveness': 0.109,
  'valence': 0.242,
  'tempo': 140.096,
  'time_signature': 4,
  'track_genre': 'piano',
  'is_liked': 1},
 {'track_id': '72gv4zhNvRVdQA0eOenCal',
  'artists': 'Clean Bandit;Zara Larsson',
  'album_name': 'What Is Love? (Deluxe Edition)',
  'track_name': 'Symphony (feat. Zara Larsson)',
  'popularity': 76,
  'duration_ms': 212733,
  'explicit': False,
  'danceability': 0.715,
  'energy': 0.605,
  'key': 0,
  'loudness': -5.128,
  'mode': 0,
  'speechiness': 0.0428,
  'acousticness': 0.239,
  'instrumentalness': 1.41e-05,
  'liveness': 0.189,
  'valence': 0.454,
  'tempo': 122.956,
  'time_signature': 4,
  't

In [26]:
import json

with open('./data/sample_for_predict_test.json', 'w') as fp:
    json.dump(samples, fp)

In [27]:
!python predict-test.py

{"predictions":[{"song":"If I Lose Myself -OneRepublic","decision":true,"probability":0.907,"description":"This song match with my taste! Recommended to be saved in my favourite song list"},{"song":"Symphony (feat. Zara Larsson) -Clean Bandit;Zara Larsson","decision":true,"probability":1.0,"description":"This song match with my taste! Recommended to be saved in my favourite song list"},{"song":"あの夢をなぞって -YOASOBI","decision":true,"probability":1.0,"description":"This song match with my taste! Recommended to be saved in my favourite song list"},{"song":"Love on the Weekend -John Mayer","decision":true,"probability":0.998,"description":"This song match with my taste! Recommended to be saved in my favourite song list"},{"song":"Love on the Weekend -John Mayer","decision":true,"probability":0.999,"description":"This song match with my taste! Recommended to be saved in my favourite song list"},{"song":"Honky Tonkin' -George Jones","decision":false,"probability":0.0,"description":"Well, good 

## Building Docker Image

In [31]:
!cat Dockerfile

FROM python:3.9.12-slim

RUN pip install pipenv

WORKDIR /app

COPY ["Pipfile", "Pipfile.lock", "./"]

# install the pipenv dependencies for the project and deploy them.
RUN pipenv install --deploy --system

# Copy any python files and the model we had to the working directory of Docker 
RUN apt update && apt install libgomp1 -y
COPY ["predict.py", "./"]
COPY ["model/", "./model"]


# We need to expose the 8080 port because we're not able to communicate with Docker outside it
EXPOSE 8080

# If we run the Docker image, we want our app to be running
ENTRYPOINT ["uvicorn", "predict:app", "--host", "0.0.0.0", "--port", "8080"]


In [32]:
!docker build -t desenfirman/spotify-personal-song-recommendation .

[+] Building 0.0s (0/1)                                                         
[+] Building 0.1s (2/3)                                                         
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 676B                                       0.0s
 => [internal] load metadata for docker.io/library/python:3.9.12-slim      0.1s
[+] Building 0.3s (2/3)                                                         
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 676B                                       0.0s
 => [internal] load metadata for dock

We will run this command on terminal to serve a model. We will use the docker's bridge IP to connect with the served model

```bash
docker run --rm --name spotify-personal-song-recommendation -t desenfirman/spotify-personal-song-recommendation
```

In [33]:
import requests
import json


url = "http://172.17.0.3:8080/batch-predict" # Use Docker Bridge IP instead

with open('./data/sample_for_predict_test.json', 'r') as fp:
    payload = json.load(fp)
    headers = {
        'Content-Type': 'application/json'
    }

    response = requests.request("POST", url, headers=headers, json=payload)

    print(response.text)

{"predictions":[{"song":"If I Lose Myself -OneRepublic","decision":true,"probability":0.907,"description":"This song match with my taste! Recommended to be saved in my favourite song list"},{"song":"Symphony (feat. Zara Larsson) -Clean Bandit;Zara Larsson","decision":true,"probability":1.0,"description":"This song match with my taste! Recommended to be saved in my favourite song list"},{"song":"あの夢をなぞって -YOASOBI","decision":true,"probability":1.0,"description":"This song match with my taste! Recommended to be saved in my favourite song list"},{"song":"Love on the Weekend -John Mayer","decision":true,"probability":0.998,"description":"This song match with my taste! Recommended to be saved in my favourite song list"},{"song":"Love on the Weekend -John Mayer","decision":true,"probability":0.999,"description":"This song match with my taste! Recommended to be saved in my favourite song list"},{"song":"Honky Tonkin' -George Jones","decision":false,"probability":0.0,"description":"Well, good 

## Deploying Service to Cloud

We will use the Google Cloud Run to serve the model over http. Google Cloud Run is chosen due to fact that it's serverless. So, you can pay the the computing resource based on the usage of the endpoint

In this scenario, the docker based Google Cloud SDK tool will be used to deploying the service to the Google Cloud Run. 

First, we will need to create a container to store the GCP credentials inside the Docker container

In [35]:
# !docker rm gcloud-config
!docker run -ti --name gcloud-config gcr.io/google.com/cloudsdktool/google-cloud-cli:alpine /bin/bash -c 'echo "Successfully creating GCP cred store"'

docker: Error response from daemon: Conflict. The container name "/gcloud-config" is already in use by container "3fb671031e784c43be551caa13b983397cef3e1c9cd054764bb8c47f947809dc". You have to remove (or rename) that container to be able to reuse that name.
See 'docker run --help'.


Then, we will perform an authentication to the Google's OAuth2 endpoint.

> PS: Make sure to run this command from the terminal instead from jupyter cell, since it will needed to input the code interactively

In [36]:
!docker run --rm --volumes-from gcloud-config -ti gcr.io/google.com/cloudsdktool/google-cloud-cli:alpine /bin/bash -c 'gcloud auth login --no-launch-browser && gcloud config set project dese-gcp'

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=rLDqsUWHP4uX3oWMKdIRJvx6jNaqFU&prompt=consent&access_type=offline&code_challenge=cXhLM--nWymVWvh6rhOOK3amr3uMeRAaISKnqwP3jrc&code_challenge_method=S256

Enter authorization code: ^C


Command killed by keyboard interrupt



Check if the project is detected after authentication

In [37]:
!docker run --rm --volumes-from gcloud-config -ti gcr.io/google.com/cloudsdktool/google-cloud-cli:alpine gcloud projects list | grep dese-gcp

dese-gcp                     dese-gcp                     191258647684


Run this command to tag and push the image into the Google Container Registry. Make sure to change the location (in this case: `asia-southeast-2`), project_id (in this case: `dese-gcp`) and source image name (in this case: `desenfirman/spotify-personal-song-recommendation`) into your own value

![Docker Push](pictures/docker_push.png)

In [ ]:
!docker run \
    --rm \
    --volumes-from gcloud-config \
    -v /var/run/docker.sock:/var/run/docker.sock \
    -ti gcr.io/google.com/cloudsdktool/google-cloud-cli:alpine \
    /bin/bash -c "gcloud auth configure-docker asia-southeast2-docker.pkg.dev && docker tag desenfirman/spotify-personal-song-recommendation asia-southeast2-docker.pkg.dev/dese-gcp/cloud-run-source-deploy/spotify-personal-song-recommendation && docker push asia-southeast2-docker.pkg.dev/dese-gcp/cloud-run-source-deploy/spotify-personal-song-recommendation"

Run this command to deploy the uploaded image into Google Cloud Run. This command will deploy the related image with the service name `spotify-personal-song-recommendation`

![Deploy Service](pictures/deploy_cloud_run.png)

In [ ]:
!docker run \
    --rm \
    --volumes-from gcloud-config \
    -v /var/run/docker.sock:/var/run/docker.sock \
    -ti gcr.io/google.com/cloudsdktool/google-cloud-cli:alpine \
    /bin/bash -c 'gcloud run deploy spotify-personal-song-recommendation --region asia-southeast2 --allow-unauthenticated --image asia-southeast2-docker.pkg.dev/dese-gcp/cloud-run-source-deploy/spotify-personal-song-recommendation'

## Test the Cloud deployment

Check the home `README.md` file